# Tutorial 5 - Symbolic Pursuit

In this tutorial we we create a symbolic pursuit explainer object and use it to get an explanation of the predictive model. The explainer is then saved to disk and can be given to someone else to view in the [Interpretability Suite App](https://vanderschaarlab-demo-interpretabi-interpretability-suite-1uteyn.streamlit.app/).

We will be explaining the predictions of a multilayer perceptron regressor provided by sci-kit learn that is trained on the diabetes dataset also from sci-kit learn.

### Import the relevant modules

In [14]:
# IMPORTS

# Third Party
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor#
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression#
from sklearn.metrics import mean_squared_error

# Interpretability
from interpretability.interpretability_models import symbolic_pursuit_explainer
from interpretability.interpretability_models.utils import io

### Load the data 
Load the data and split it into the different sets for training the mlp, fitting the explainer and testing the explainer.

In [15]:

X, y = load_diabetes(return_X_y=True, as_frame=True)

feature_names = X.columns.to_list()
X_mlp_train, X_test, y_mlp_train, y_test = train_test_split(X, y, test_size=0.5)
X_expl_train, X_explain_test, y_expl_train, y_explain_test = train_test_split(X_test, y_test, test_size=0.2)

X_mlp_train, y_mlp_train, X_expl_train, y_expl_train, X_explain_test, y_explain_test = (
    X_mlp_train.to_numpy(),
    y_mlp_train.to_numpy(),
    X_expl_train.to_numpy(),
    y_expl_train.to_numpy(),
    X_explain_test.to_numpy(),
    y_explain_test.to_numpy(),
)

### Train the model

We will train our own model here using the sklearn library here. We simply have to initialize it and fit it, before it is ready to pass to the SymbolicPursuitExplainer in the next step.

In [16]:
## Load the model

model = LinearRegression()
model.fit(X_mlp_train, y_mlp_train)
model.score(X_explain_test, y_explain_test)

### Initialize and fit Symbolic Pursuit Explainer
Initialize the explainer object by passing the models predict function, and the data to explain. The fit step can take some considerable time, this can be reduced by reducing the `patience` argument. The default value for patience is 10, reducing it significantly from that value, may affect the quality of the explanation.

In [17]:

my_explainer = symbolic_pursuit_explainer.SymbolicPursuitExplainer(
    model.predict, X_expl_train, feature_names=feature_names, patience=1
)
my_explainer.fit()

### Measure fit quality
This prints the Mean Squared Error for both the mlp predictive model and the learned symbolic model

In [18]:
my_explainer.measure_fit_quality(X_explain_test, y_explain_test)

MSE score for the model: 3400.9267047529574
MSE score for the Symbolic Regressor: 3913.308267786906


### Get the explanation
Get the explanation of the model in terms of its symbolic expression and projections of the predictive model.

In [19]:
explanation = my_explainer.explain(X_explain_test[3], taylor_expansion_order=2)

0


### Show the symbolic expression and projections

In [20]:
my_explainer.summary_plot(show_expression=False, show_feature_importance=True, show_feature_interactions=True)

1.28673528875237*meijerg(((-0.0238334588166228,), (0.677142219058804,)), ((-1.89775855719856, -0.76336887618189), (-0.434727427351685,)), 1.0*[ReLU(P1)])
P1 = -0.261979271495097*X1 - 0.568545152086398*X10 + 0.884542293937177*X2 - 1.89124887830286*X3 - 0.840798636983763*X4 - 0.0968543199716866*X5 + 0.700029016346244*X6 + 1.11613582817466*X7 + 0.113530407585627*X8 - 2.18748153588367*X9



,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


The taylor expansion that calculates feature interactions is not available. Try fitting the SymbolicPursuitExplainer for more iterations, by increasing `patience` or reducing `loss_tol`.


In [24]:
print(explanation.feature_importance)
print(my_explainer.symbolic_model.predict(X_explain_test[1]))

[1e-05, 1e-05, 1e-05, 1e-05, 1e-05, 1e-05, 1e-05, 1e-05, 1e-05, 1e-05]
[201.90674082 201.90674082 201.90674082 201.90674082 201.90674082
 201.90674082 201.90674082 201.90674082 201.90674082 201.90674082]


In [27]:
print(explanation.taylor_expansion)

0


### Save the explainer to file
This file can now be uploaded to the [Interpretability Suite App](https://vanderschaarlab-demo-interpretabi-interpretability-suite-1uteyn.streamlit.app/). This provides a non-programtic interface with which to view the various explanations, allowing you to send the explainer to a colleague who is less fluent in python.

In [ ]:

io.save_explainer(
    my_explainer, "my_new_diabetes_sklearn_random_forrest_symbolic_pursuit_explainer.p"
)

Saving explainer to: /home/rob/Documents/projects/Interpretability/Notebooks/my_new_diabetes_sklearn_random_forrest_symbolic_pursuit_explainer.p
